In [8]:
import pandas as pd

In [9]:
#reading data from the Training_data.csv as a dataframe

df = pd.read_csv("/Users/shreejavasala/Documents/Mini_Project/Training_data.csv")
print(df.shape)
df.head(10)

(231, 2)


,Text,Category
0,"Hi, How are you?",Conversation
1,Open the Calculator,Automation
2,What is Computer Science?,Query
3,"Hello, How is it going",Conversation
4,Is it going to rain today in Mumabai?,Query
5,Thank you for helping me out today,Conversation
6,Hi there!,Conversation
7,please make a call to my friend,Automation
8,Send an e-mail to my colleague,Automation
9,Back up recently deleted files,Automation


In [10]:
df['Category'].unique()
df['Category'].value_counts()

Category
Query           84
Automation      78
Conversation    69
Name: count, dtype: int64

In [11]:
df['Category_Vector'] = df['Category'].map({
    'Conversation' : 0,
    'Automation' : 1,
    'Query' : 2
})
df.head()
print(df['Category_Vector'].values)

[0 1 2 0 2 0 0 1 1 1 2 2 2 2 0 2 2 2 0 1 1 2 2 0 2 2 1 1 1 0 0 0 0 2 2 0 1
 2 1 2 1 0 2 2 2 0 0 2 1 1 2 1 1 1 1 1 0 0 0 2 1 1 0 2 1 1 1 1 1 0 0 0 0 0
 0 0 0 1 1 1 2 2 2 2 2 2 1 1 1 1 1 0 0 0 0 0 0 1 2 1 1 1 2 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 1 1 1 1
 1 1 1 1 2 2 2 1 0 0 1 0 2 0 0 2 2 2 1 1 2 1 2 1 2 1 2 1 1 0 2 0 0 1 0 1 2
 2 2 1 1 1 2 2 1 1]


In [12]:
import spacy
nlp = spacy.load("en_core_web_md")

In [13]:
df['Text_Vector'] = df['Text'].apply(lambda text: nlp(' '.join([token.text for token in nlp(text) if not token.is_punct])).vector)


In [14]:
df.head()

,Text,Category,Category_Vector,Text_Vector
0,"Hi, How are you?",Conversation,0,"[-0.69876504, -0.0001500044, -0.19010301, -0.4..."
1,Open the Calculator,Automation,1,"[-0.64370006, 0.13723333, -0.115854345, 0.2468..."
2,What is Computer Science?,Query,2,"[-0.69642, 0.33735198, -0.13490349, -0.1035250..."
3,"Hello, How is it going",Conversation,0,"[-0.68201, 0.23253398, -0.42377, -0.18508801, ..."
4,Is it going to rain today in Mumabai?,Query,2,"[-0.62227756, 0.2895575, -0.213225, -0.0321208..."


In [15]:
#training the model using train_test_split with 20% sample data

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
                                    df.Text_Vector.values,
                                    df.Category_Vector,
                                    test_size = 0.2,
                                    random_state = 500)  
print(X_train.shape)
print(X_test.shape)
# print(df['Text_Vector'].shape)
# print(y_train)

(184,)
(47,)


In [16]:
import numpy as np
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [17]:
# scaling

from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
scaled_train_embed = scalar.fit_transform(X_train_2d)
scaled_test_embed = scalar.transform(X_test_2d)

In [18]:
# Naive_bayes model - working comparitively better in this case

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
 
mb = MultinomialNB()
mb.fit(scaled_train_embed, y_train)
y_pred = mb.predict(scaled_test_embed)
print("Classification Report: ")
target_names = df['Category'].unique().tolist()
print(classification_report(y_pred,y_test, target_names = target_names, zero_division = 0))

Classification Report: 
              precision    recall  f1-score   support

Conversation       0.93      0.67      0.78        21
  Automation       0.79      0.65      0.71        17
       Query       0.44      0.89      0.59         9

    accuracy                           0.70        47
   macro avg       0.72      0.73      0.69        47
weighted avg       0.79      0.70      0.72        47



In [19]:
#KNN model

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(scaled_train_embed, y_train)
y_pred = knn.predict(scaled_test_embed)
target_names = df['Category'].unique().tolist()
print(f"Classification Report: {classification_report(y_pred,y_test, target_names = target_names, zero_division = 0)}")

Classification Report:               precision    recall  f1-score   support

Conversation       0.93      0.64      0.76        22
  Automation       0.71      0.71      0.71        14
       Query       0.50      0.82      0.62        11

    accuracy                           0.70        47
   macro avg       0.72      0.72      0.70        47
weighted avg       0.77      0.70      0.71        47



In [20]:
#SVM model

from sklearn.svm import SVC
svc = SVC()
svc.fit(scaled_train_embed, y_train)
y_pred = svc.predict(scaled_test_embed)
print("Classification Report: ")
target_names = df['Category'].unique().tolist()
print(classification_report(y_pred,y_test, target_names = target_names, zero_division = 0))

Classification Report: 
              precision    recall  f1-score   support

Conversation       0.93      0.74      0.82        19
  Automation       0.79      0.58      0.67        19
       Query       0.44      0.89      0.59         9

    accuracy                           0.70        47
   macro avg       0.72      0.73      0.69        47
weighted avg       0.78      0.70      0.72        47



In [21]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
lg.fit(scaled_train_embed, y_train)
y_pred = lg.predict(scaled_test_embed)
print("Classification Report: ")
target_names = df['Category'].unique().tolist()
print(classification_report(y_pred, y_test, target_names = target_names, zero_division = 0))


Classification Report: 
              precision    recall  f1-score   support

Conversation       0.87      0.72      0.79        18
  Automation       0.79      0.85      0.81        13
       Query       0.72      0.81      0.76        16

    accuracy                           0.79        47
   macro avg       0.79      0.79      0.79        47
weighted avg       0.80      0.79      0.79        47



/Users/shreejavasala/Documents/Mini_Project/venv_mini/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:


#Categorizing the user_input

# def preprocess_user_input(input):
#         doc = nlp(input)
#         tokens = [token.text for token in doc if not token.is_punct] #removes the punctuation
#         # print((tokens)) -->list of tokens of type string
#         n_output = ' '.join(tokens)
#         # print(n_output) -->string
#         doc = nlp(n_output)
#         n_output = doc.vector
#         # print(n_output.shape) --> 1D vector
#         n_output = doc.reshape(1,-1)
#         # print(n_output.shape) --> 2D array(vector) after reshape
#         scaled_input_embedding = scalar.transform(n_output)
#         prediction = mb.predict(scaled_input_embedding)
#         category_map = {0: "Conversation", 1: "Automation", 2: "Query"}
#         predicted_Category = category_map[prediction[0]]
#         return predicted_Category


def preprocess_user_input(input):
        doc = nlp(input)
        input_vector = [token.vector for token in doc if not token.is_punct] #removes the punctuation
        if not input_vector:
                return "unknown category"    
        n_output = np.stack(input_vector)
        scaled_input_embedding = scalar.transform(n_output)
        prediction = svc.predict(scaled_input_embedding)
        category_map = {0: "Conversation", 1: "Automation", 2: "Query"}
        predicted_Category = category_map[prediction[0]]
        return predicted_Category


user_input= input(("Enter your Text: "))
print("Input: ", user_input)
print("Category: ", preprocess_user_input(user_input))

Input:  how's day going?
Category:  Conversation
